In [ ]:
! pip install ax-platform --quiet
! pip install --upgrade git+https://github.com/cornellius-gp/gpytorch.git
! wget https://raw.githubusercontent.com/chuckwong13/Ax/master/ax/utils/tutorials/cnn_utils.py
! pip install livelossplot --quiet
! pip install scikit-misc --quiet
!  pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html

     |████████████████████████████████| 501kB 2.8MB/s 
     |████████████████████████████████| 225kB 13.5MB/s 
     |████████████████████████████████| 245kB 11.6MB/s 
  Cloning https://github.com/cornellius-gp/gpytorch.git to /tmp/pip-req-build-amny2tzp
  Running command git clone -q https://github.com/cornellius-gp/gpytorch.git /tmp/pip-req-build-amny2tzp
  Created wheel for gpytorch: filename=gpytorch-1.1.1-py2.py3-none-any.whl size=449287 sha256=80ba23445f8b2405fa3bae4eef7b5d344bbe27605a6a906ee65ef6e04539369b
  Stored in directory: /tmp/pip-ephem-wheel-cache-7s3_mh59/wheels/52/ac/ca/6dc889cad19909fc1590b26613028a9c8df1a4baf9b7385002
Successfully built gpytorch
  Found existing installation: gpytorch 1.1.1
    Uninstalling gpytorch-1.1.1:
      Successfully uninstalled gpytorch-1.1.1
--2020-08-10 05:25:44--  https://raw.githubusercontent.com/chuckwong13/Ax/master/ax/utils/tutorials/cnn_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.

In [ ]:
from copy import deepcopy
import numpy as np
import pandas as pd
from scipy.stats import norm
import time

from ax.core.data import Data
from ax.core.observation import ObservationFeatures, observations_from_data
from ax.core.optimization_config import OptimizationConfig
from ax.core.search_space import SearchSpace
from ax.core.objective import Objective
from ax.runners.synthetic import SyntheticRunner
from ax.modelbridge.random import RandomModelBridge
from ax.core.outcome_constraint import OutcomeConstraint
from ax.core.types import ComparisonOp
from ax.core.parameter import RangeParameter, ParameterType
from ax.core.multi_type_experiment import MultiTypeExperiment
from ax.metrics.hartmann6 import Hartmann6Metric
from ax.metrics.l2norm import L2NormMetric
from ax.modelbridge.factory import get_sobol, get_GPEI, get_MTGP
from ax.core.generator_run import GeneratorRun
from ax.plot.diagnostic import interact_batch_comparison
from ax.plot.trace import optimization_trace_all_methods
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax import *

import torch
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting
from cnn_utils import load_mnist, train, evaluate, CNN, load_usps, CNN_usps
import matplotlib.pyplot as plt
import torch.optim as optim

# Generate Data for CNN

##Load Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(12345)
dtype = torch.float

BATCH_SIZE = 512
train_loader_mnist, valid_loader_mnist, test_loader_mnist = load_mnist(batch_size=BATCH_SIZE)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning:

The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)



In [ ]:
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


##Load Model

In [ ]:
def train_evaluate_mnist(parameterization):
        net = CNN()
        net = train(net=net, train_loader=train_loader_mnist, parameters=parameterization, dtype=dtype, device=device)
        return evaluate(
            net=net,
            data_loader=valid_loader_mnist,
            dtype=dtype,
            device=device,
        )

##Define variable to tune

In [ ]:
parameters=[
        RangeParameter(
            name="lr", parameter_type=ParameterType.FLOAT, lower=0.0001, upper=0.1, log_scale = True),
        RangeParameter(
            name="eps", parameter_type=ParameterType.FLOAT, lower=0, upper=1e-8),
        RangeParameter(
            name="beta1", parameter_type=ParameterType.FLOAT, lower=0, upper=0.999),
        RangeParameter(
            name="beta2", parameter_type=ParameterType.FLOAT, lower=0, upper=0.999), 
        RangeParameter(
            name="weight_decay", parameter_type=ParameterType.FLOAT, lower=0, upper=0.3),       
        RangeParameter(
            name="num_epochs", lower=1, upper=15, parameter_type=ParameterType.INT)
    ]
search_space = SearchSpace(
    parameters=parameters
) 

## Generate Data

In [ ]:
n = 500
sobol = Models.SOBOL(search_space=search_space)
generator_run = sobol.gen(n)

x = pd.DataFrame()
y = []
i = 0
for arm in generator_run.arms:
    print(i)
    x = x.append(arm.parameters,ignore_index=True)
    # y = df.append(train_evaluate_mnist(parameterization=arm.parameters),ignore_index=True)
    y.append(train_evaluate_mnist(parameterization=arm.parameters))
    i += 1

In [ ]:
x.to_csv('X.csv')
!cp X.csv "gdrive/My Drive/MTBO for Hyperparameters of CNN"

In [ ]:
name = ['acc',]
Y=pd.DataFrame(columns=name,data=y)
Y.to_csv('Y.csv')
!cp Y.csv "gdrive/My Drive/MTBO for Hyperparameters of CNN"

# Generate data for Hartmann6

In [ ]:
def hartmann6(x):
  _required_dimensionality = 6
  _domain = [(0, 1) for i in range(6)]
  _minimums = [(0.20169, 0.150011, 0.476874, 0.275332, 0.311652, 0.6573)]
  _fmin = -3.32237
  _fmax = 0.0
  _alpha = np.array([1.0, 1.2, 3.0, 3.2])
  _A = np.array(
      [
          [10, 3, 17, 3.5, 1.7, 8],
          [0.05, 10, 17, 0.1, 8, 14],
          [3, 3.5, 1.7, 10, 17, 8],
          [17, 8, 0.05, 10, 0.1, 14],
      ]
  )
  _P = 10 ** (-4) * np.array(
      [
          [1312, 1696, 5569, 124, 8283, 5886],
          [2329, 4135, 8307, 3736, 1004, 9991],
          [2348, 1451, 3522, 2883, 3047, 6650],
          [4047, 8828, 8732, 5743, 1091, 381],
      ]
  )
  X = list(x.values())
  y = 0.0
  for j, alpha_j in enumerate(_alpha):
      t = 0
      for k in range(6):
          t += _A[j, k] * ((X[k] - _P[j, k]) ** 2)
      y -= alpha_j * np.exp(-t)
  return float(y)

In [ ]:
parameters=[
    RangeParameter(
        name=param_names[i], parameter_type=ParameterType.FLOAT, lower=0.0, upper=1.0
    )
    for i in range(10)
]
search_space=SearchSpace(parameters=parameters)

In [ ]:
n = 2000
sobol = Models.SOBOL(search_space=search_space)
generator_run = sobol.gen(n)

x1 = pd.DataFrame()
y1 = []
for arm in generator_run.arms:
    x1 = x1.append(arm.parameters,ignore_index=True)
    # y = df.append(train_evaluate_mnist(parameterization=arm.parameters),ignore_index=True)
    y1.append(hartmann6(arm.parameters))

In [ ]:
x1.to_csv('X_hartmann.csv')
!cp X_hartmann.csv "gdrive/My Drive/MTBO for Hyperparameters of CNN"

In [ ]:
name = ['y',]
Y1=pd.DataFrame(columns=name,data=y1)
Y1.to_csv('Y_hartmann.csv')
!cp Y_hartmann.csv "gdrive/My Drive/MTBO for Hyperparameters of CNN"